In [3]:
# %load mnist.py
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

import torch
import nn
import nn.functional as F


n_features = 28 * 28    # 每个数据的大小形状
n_classes = 10    # 最后分为的种类数
n_epochs = 10     # 训练10次
bs = 1000     # 每次的批量数
lr = 1e-3     # 学习率
lengths = (n_features, 512, n_classes)    # 512：Linear隐藏层数据维度,  nfeatures:输入数据的维度形状，nclasses：输出种类数


class Model(nn.Module):

    # TODO Design the classifier.
    def __init__(self, lengths):
        self.n_features, self.hidden_dims, self.n_classes = lengths
        self.conv1 = nn.Conv2d(1, 16)    # 16个滤波器，通道数为1（黑白图片）
        self.bn1 = torch.nn.BatchNorm2d(16)
        self.relu1 = F.Relu()
        self.pool1 = nn.MaxPool(2, stride=2, pad=0)
        self.conv2 = nn.Conv2d(16, 32)
        self.bn2 = torch.nn.BatchNorm2d(32)
        self.relu2 = nn.Relu()
        self.pool2 = nn.MaxPool(2, stride=2, pad=0)
        
        self.fc1 = nn.Linear(32 * 4 * 4, 64)
        self.relu3 = nn.Relu()
        self.fc2 = nn.Linear(64, 10)
        
        
        # self.loss = SoftmaxWithLoss()    这个东西放到optimizer。
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x
        
    
    def backward(self, dout):
        dx = self.fc2.backward(dout)
        dx = self.relu3.backward(dx)
        dx = self.fc1.backward(dx)
        dx = self.pool2.backward(dx)
        dx = self.relu2.backward(dx)
        dx = self.bn2.backward(dx)
        dx = self.conv2.backward(dx)
        dx = self.pool1.backward(dx)
        dx = self.relu1.backward(dx)
        dx = self.bn1.backward(dx)
        dx = self.conv1.backward(dx)
        
        return dx
        
    ...

    # End of todo


def load_mnist(mode='train', n_samples=None):
    images = '././././../././././../Data/MNIST/raw/train-images-idx3-ubyte' if mode == 'train' else '././././../././././../Data/MNIST/raw/t10k-images-idx3-ubyte'
    labels = '././././../././././../Data/MNIST/raw/train-labels-idx1-ubyte' if mode == 'train' else '././././../././././../Data/MNIST/raw/t10k-labels-idx1-ubyte'
    length = 60000 if mode == 'train' else 10000

    X = np.fromfile(open(images), np.uint8)[16:].reshape((length, 28, 28)).astype(np.int32)
    y = np.fromfile(open(labels), np.uint8)[8:].reshape((length)).astype(np.int32)
    return (X[:n_samples].reshape(n_samples, -1), y[:n_samples]) if n_samples is not None else (X.reshape(length, -1), y)


def vis_demo(model):
    X, y = load_mnist('test', 20)
    probs = model.forward(X)
    preds = np.argmax(probs, axis=1)
    fig = plt.subplots(nrows=4, ncols=5, sharex='all', sharey='all')[1].flatten()
    for i in range(20):
        img = X[i].reshape(28, 28)
        fig[i].set_title(preds[i])
        fig[i].imshow(img, cmap='Greys', interpolation='nearest')
    fig[0].set_xticks([])
    fig[0].set_yticks([])
    plt.tight_layout()
    plt.savefig("vis.png")
    plt.show()


def main():
    trainloader = nn.data.DataLoader(load_mnist('train'), batch=bs)
    testloader = nn.data.DataLoader(load_mnist('test'))
    model = Model(lengths)
    optimizer = nn.optim.SGD(model, lr=lr, momentum=0.9)
    criterion = F.CrossEntropyLoss(n_classes=n_classes)

    for i in range(n_epochs):
        bar = tqdm(trainloader, total=6e4 / bs)
        bar.set_description(f'epoch  {i:2}')
        for X, y in bar:
            probs = model.forward(X)   # 实现一次前向传播
            loss = criterion(probs, y)   # 得到损失函数
            model.backward(loss.backward())  # 损失函数反向传播后令model也反向传播
            optimizer.step()               # 更新权重
            preds = np.argmax(probs, axis=1)   # 得到本次预测结果
            bar.set_postfix_str(f'acc={np.sum(preds == y) / len(y) * 100:.1f} loss={loss.value:.3f}')   # 得到训练集预测成功率

        for X, y in testloader:       # 得到测试集预测成功率
            probs = model.forward(X)
            preds = np.argmax(probs, axis=1)
            print(f' test acc: {np.sum(preds == y) / len(y) * 100:.1f}')

    vis_demo(model)


if __name__ == '__main__':
    main()


TypeError: __init__() takes 1 positional argument but 2 were given